In [ ]:
pip install keras_tuner

In [ ]:
# Import necessary libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras_tuner as kt

from keras_tuner.engine.hyperparameters import HyperParameter as hp

In [ ]:
# Define necessary functions
# Function to return searched optimizers
def return_optimizer(optimizer_search, learning_rate_search):

    # If else block to return the optimizer and learning rate
    if optimizer_search == 'adam':

        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_search)
    
    elif optimizer_search == 'nadam':

        optimizer = tf.keras.optimizers.Nadam(learning_rate=learning_rate_search)
    
    elif optimizer_search == 'rmsprop':

        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate_search)
    
    elif optimizer_search == 'adadelta':

        optimizer = tf.keras.optimizers.Adadelta(learning_rate=learning_rate_search)
    
    elif optimizer_search == 'adagrad':

        optimizer = tf.keras.optimizers.Adagrad(learning_rate=learning_rate_search)

    elif optimizer_search == 'adamax':

        optimizer = tf.keras.optimizers.Adamax(learning_rate=learning_rate_search)
    
    return optimizer

In [ ]:
# Function to save files
def save_file(values, file_name, file_directory):
    
    # Save the file as a .npy file
    np.save(os.path.join(file_directory, file_name), values)
    
    print(f'Saved {file_name} to {file_directory}')

In [ ]:
# Function to build model for hyperparameter search
def build_model(hp):

    # Model Architecture Stage
    # Filter Sizes
    filter_search = hp.Int('num_of_filters', min_value = 100, max_value = 1000, step = 100)

    # Input layer
    input_layer = tf.keras.layers.Input(shape=input_shape)
    
    # Convolutional Layers
    # First Convolutional Layer
    x1 = tf.keras.layers.Conv1D(filters=filter_search,
                                kernel_size=32,
                                padding='same',
                                kernel_initializer='glorot_normal',
                                activation='LeakyReLU')(input_layer)
    
    # Batch Normalization Layer
    x1 = tf.keras.layers.BatchNormalization()(x1)

    # Define the skip connection
    skip_connection_1 = x1

    # Second Convolutional Layer
    x2 = tf.keras.layers.Conv1D(filters=filter_search,
                                kernel_size=32,
                                padding='same',
                                kernel_initializer='glorot_normal',
                                activation='LeakyReLU')(x1)
    
    # Batch Normalization Layer
    x2 = tf.keras.layers.BatchNormalization()(x2)

    # Add the skip connection connection
    x2 = tf.keras.layers.Add()([skip_connection_1, x2])

    # Define the skip connection
    skip_connection_2 = x2
    
    # Third Convolutional Layer
    x3 = tf.keras.layers.Conv1D(filters=filter_search,
                                kernel_size=32,
                                padding='same',
                                kernel_initializer='glorot_normal',
                                activation='LeakyReLU')(x2)
    
    # Batch Normalization Layer
    x3 = tf.keras.layers.BatchNormalization()(x3)

    # Add the skip connection connection
    x3 = tf.keras.layers.Add()([skip_connection_2, x3])

    # Output Layer
    output_layer = tf.keras.layers.Conv1D(filters=output_shape[1], kernel_size=32, padding='same', activation='LeakyReLU')(x3)
    
    # Define the model
    model = tf.keras.models.Model(inputs=[input_layer], outputs=[output_layer])

    # Compile the model
    model.compile(optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001), 
                  loss = 'mae', 
                  metrics = 'mse')
    
    # Return the model
    return model

In [ ]:
# Define directories
current_directory = os.getcwd()

# Define directory for the normalized data
normalized_data_directory = os.path.join(current_directory, '..', '..', 'data', 'normalized')

# Define directory for the hyperparameter search
save_directory = os.path.join(current_directory, '..', '..', 'residual-concatenate')
hyperparameter_search_folder_name = 'hyperparameter_search_results'

In [ ]:
# Working with just the displacement data to conserve memory
# Load the normalized training subsets for displacement data
print('Loading the normalized training subsets for displacement data...')
normalized_training_displacement_data = np.load(os.path.join(normalized_data_directory, 'normalized_training_displacement_data.npy'))

In [ ]:
# Load the normalized training subsets for force data
print('Loading the normalized training subsets for force data...')
normalized_training_force_data = np.load(os.path.join(normalized_data_directory, 'normalized_training_force_data.npy'))

In [ ]:
# Print the shapes of the displacement data
print(f'The shape of displacement data is {normalized_training_displacement_data.shape[1:]}.')
print(f'The shape of force data is {normalized_training_force_data.shape[1:]}.')

In [ ]:
# Define variables that remain constant during the training
input_shape = normalized_training_displacement_data.shape[1:]
output_shape = normalized_training_force_data.shape[1:]

In [ ]:
# Use the hyperband tuner from keras tuner
hyperband_tuner = kt.Hyperband(build_model,
                                objective = kt.Objective('val_mse', direction = 'min'),
                                max_epochs = 500,
                                directory = save_directory,
                                project_name = 'hyperband_search_results')

In [ ]:
# Define callbacks
# Early stopping callback
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)

In [ ]:
# Display search space summary
hyperband_tuner.search_space_summary()

In [ ]:
# Split 80-20 into training and validation sets
normalized_training_displacement_data = normalized_training_displacement_data[:int(0.8 * normalized_training_displacement_data.shape[0])]
normalized_training_force_data = normalized_training_force_data[:int(0.8 * normalized_training_force_data.shape[0])]

normalized_validation_displacement_data = normalized_training_displacement_data[int(0.8 * normalized_training_displacement_data.shape[0]):]
normalized_validation_force_data = normalized_training_force_data[int(0.8 * normalized_training_force_data.shape[0]):]

In [ ]:
# Search for the best hyperparameters
hyperband_tuner.search(normalized_training_displacement_data, normalized_training_force_data,
                        epochs = 100,
                        validation_data = (normalized_validation_displacement_data, normalized_validation_force_data),
                        callbacks = [early_stopping_callback])

In [ ]:
# Get the best trained model
best_model = hyperband_tuner.get_best_models(num_models = 1)[0]

# Save the best model
best_model.save(os.path.join(save_directory, 'training_results', 'best_model.h5'))

# Print message to the user
print('Hyperparameter search completed successfully.')